# 6 Naive collaborative filtering

# Q30

In [11]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
import os
#from surprise import Trainset
from six import iteritems
from surprise.model_selection.split import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# path to dataset file
file_path = os.path.expanduser('~/Desktop/EE219A/ml-latest-small/ratings.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=1)

data = Dataset.load_from_file(file_path, reader=reader)


class NaiveFilter(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)
        
    def fit(self, trainset):

        AlgoBase.fit(self, trainset)
        self.means = np.zeros(self.trainset.n_users)
        
        for x, ratings in iteritems(self.trainset.ur):
            self.means[x] = np.mean([r for (_, r) in ratings])

        return self


    def estimate(self, u, i):
        est = self.means[u]

        return est



kf = KFold(n_splits=10)
algo = NaiveFilter()
trainset = data.build_full_trainset()
algo.fit(trainset)
rmse_nf = []
for trainset_new, testset in kf.split(data):

    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    #accuracy.rmse(predictions, verbose=True)
    rmse_nf.append(accuracy.rmse(predictions, verbose=True))

RMSE: 0.9437
RMSE: 0.9732
RMSE: 0.9432
RMSE: 0.9521
RMSE: 0.9570
RMSE: 0.9683
RMSE: 0.9576
RMSE: 0.9623
RMSE: 0.9419
RMSE: 0.9545


In [12]:
np.mean(rmse_nf)

0.95538232560904712

# Q31

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


data_pd = pd.read_csv('ml-latest-small/ratings.csv', sep=',')

print(len(data_pd))
data_pd.head()

100004


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
data_popular = data_pd.groupby('movieId').filter(lambda x: len(x) > 2)
data_unpopular = data_pd.groupby('movieId').filter(lambda x: len(x) <= 2)
data1 =data_pd.groupby('movieId').filter(lambda x: len(x) >= 5)
data_highvar=data1.groupby('movieId').filter(lambda x: np.var(x['rating']) >= 2 )

In [9]:
reader = Reader()
dataset_popular = Dataset.load_from_df(data_popular[['userId', 'movieId', 'rating']], reader)
dataset_unpopular = Dataset.load_from_df(data_unpopular[['userId', 'movieId', 'rating']], reader)
dataset_highvar = Dataset.load_from_df(data_highvar[['userId', 'movieId', 'rating']], reader)

In [14]:


algo = NaiveFilter()

def get_rmse(dataset):
    kf = KFold(n_splits=10)
    data = dataset
    trainset = data.build_full_trainset()
    algo.fit(trainset)
    rmse_nf = []
    for trainset_new, testset in kf.split(data):

        predictions = algo.test(testset)

        rmse_nf.append(accuracy.rmse(predictions, verbose=False))
        r = np.mean(rmse_nf)
        return r
    
popular_rmse = get_rmse(dataset_popular)
unpopular_rmse = get_rmse(dataset_unpopular)
highvar_rmse = get_rmse(dataset_highvar)

print(popular_rmse, unpopular_rmse, highvar_rmse)


0.95421186468 0.886063606432 0.932509291971
